# Interactive Python Lessons: From Turtle to 3D & Data Dashboards

## 1. Interactive Turtle Hand-Pump Drawer

**Learning Objective:** Understand how to link UI widgets to a Python function to create a responsive, graphical application. This example uses sliders to control the parameters of a drawing made with Python's `turtle` graphics library.

In [ ]:
import turtle
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# Create an output widget to hold the turtle canvas
out = widgets.Output(layout={'border': '1px solid black', 'width': '500px', 'height': '400px'})

# Define sliders for controlling the hand pump's dimensions
base_slider = widgets.IntSlider(value=100, min=50, max=200, step=10, description='Base Width:')
height_slider = widgets.IntSlider(value=150, min=50, max=300, step=10, description='Post Height:')

def draw_hand_pump(base_width, post_height):
    """Clears the canvas and redraws a simple hand pump using turtle graphics."""
    # Set up the turtle screen inside the output widget
    canvas = turtle.Screen()
    canvas.clear()
    canvas.screensize(480, 380)
    t = turtle.RawTurtle(canvas)
    t.hideturtle()
    t.speed(0) # Fastest drawing speed

    # --- Drawing logic ---
    t.penup()
    t.goto(-base_width / 2, -150)
    t.pendown()
    
    # Draw base
    t.fillcolor("gray")
    t.begin_fill()
    t.forward(base_width)
    t.left(90)
    t.forward(20)
    t.left(90)
    t.forward(base_width)
    t.left(90)
    t.forward(20)
    t.end_fill()
    
    # Draw vertical post
    t.penup()
    t.goto(0, -130)
    t.pendown()
    t.fillcolor("darkred")
    t.begin_fill()
    t.left(90)
    t.forward(post_height)
    t.right(90)
    t.forward(10)
    t.right(90)
    t.forward(post_height)
    t.end_fill()

    # Draw handle
    t.penup()
    t.goto(10, -130 + post_height)
    t.pendown()
    t.left(160)
    t.forward(100)

def on_value_change(change):
    """Callback function that is triggered when a slider's value changes."""
    with out:
        clear_output(wait=True)
        draw_hand_pump(base_slider.value, height_slider.value)

# Link the callback function to the sliders
base_slider.observe(on_value_change, names='value')
height_slider.observe(on_value_change, names='value')

# Initial drawing
with out:
    draw_hand_pump(base_slider.value, height_slider.value)

# Display the widgets in a vertical box layout
display(widgets.VBox([base_slider, height_slider, out]))

## 2. VPython 3D Water-Pump Simulator

**Learning Objective:** Explore 3D graphics and animation loops. This example uses `vpython` to create a simple 3D scene of a water pump and animates it based on a slider's value. It introduces the concept of an animation loop that continuously updates the scene.

In [ ]:
from vpython import canvas, cylinder, vector, rate, sphere, color
import ipywidgets as widgets
from IPython.display import display
import asyncio

# Set up the VPython canvas to be embedded in the notebook
scene = canvas(width=600, height=400, background=color.cyan)

# Create the 3D objects for the pump
pivot = vector(0, 0, 0)
handle = cylinder(pos=pivot, axis=vector(5, 0, 0), radius=0.2, color=color.red)
base = cylinder(pos=vector(0, -2, 0), axis=vector(0, 4, 0), radius=0.5, color=color.gray(0.7))

# Widget to control animation speed
speed_slider = widgets.FloatSlider(value=5, min=1, max=20, step=1, description='Pump Speed:')

async def update_scene():
    """An asynchronous function to run the animation loop."""
    angle = 0
    while True:
        # Control the animation frame rate
        rate(30)
        
        # Get the current speed from the slider
        speed = speed_slider.value
        angle += speed * 0.01
        
        # Rotate the handle
        handle.axis = vector(5 *_math.cos(angle), 5 *_math.sin(angle), 0)
        
        # Spawn a water droplet when the handle is at the bottom
        if 0.95 <_math.sin(angle) < 1.0:
            droplet = sphere(pos=vector(-5, 0, 0), radius=0.1, color=color.blue)
            droplet.velocity = vector(0, -1, 0)
            
            # Animate the droplet falling (this is a simplified animation)
            for _ in range(20):
                rate(30)
                droplet.pos += droplet.velocity * 0.1
            droplet.visible = False # Remove droplet after it falls
            
        await asyncio.sleep(0.01) # Allow other tasks to run

# Display the slider
display(speed_slider)

# Run the animation loop
asyncio.ensure_future(update_scene())

## 3. Pump Loop Story Dashboard

**Learning Objective:** Use loops for simulation and data collection. This example simulates collecting water with a hand pump, uses a `while` loop to reach a target, stores the results in a `pandas` DataFrame, and visualizes the data with `matplotlib`.

In [ ]:
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# --- Widgets for user input ---
liters_per_stroke_input = widgets.IntText(value=5, description='Liters/Stroke:')
target_liters_input = widgets.IntText(value=100, description='Target Liters:')
run_button = widgets.Button(description="Run Simulation", button_style='success')
input_box = widgets.HBox([liters_per_stroke_input, target_liters_input, run_button])

# --- Output widgets ---
output_df = widgets.Output()
output_plot = widgets.Output()
output_box = widgets.VBox([output_df, output_plot])

def run_simulation(b):
    """Callback for the run button. Runs the simulation and updates outputs."""
    liters_per_stroke = liters_per_stroke_input.value
    target_liters = target_liters_input.value
    
    # --- Simulation Logic ---
    water_collected = 0
    stroke_count = 0
    data = []
    while water_collected < target_liters:
        stroke_count += 1
        water_collected += liters_per_stroke
        data.append({'Stroke': stroke_count, 'Water Collected (L)': water_collected})
    
    df = pd.DataFrame(data)
    
    # --- Display DataFrame ---
    with output_df:
        clear_output(wait=True)
        print(f"It took {stroke_count} strokes to collect {water_collected} liters.")
        display(df)
        
    # --- Display Matplotlib Chart ---
    with output_plot:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.bar(df['Stroke'], df['Water Collected (L)'], color='skyblue')
        ax.set_xlabel("Stroke Number")
        ax.set_ylabel("Total Water Collected (Liters)")
        ax.set_title("Water Collection Progress")
        plt.tight_layout()
        plt.show(fig)

# Link the button to the function
run_button.on_click(run_simulation)

# Display the final layout
display(input_box, output_box)

## 4. Expert-Level Water Economy Explorer

**Learning Objective:** Work with more complex data structures (dictionaries, JSON), perform data transformations with comprehensions, and create interactive dashboards with `plotly` and tabbed layouts. This is a mini data-analysis application within the notebook.

In [ ]:
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from IPython.display import display, clear_output
import json
import math

# --- Initial Data and Widgets ---
initial_families_data = {
    "Family A": 150,
    "Family B": 80,
    "Family C": 220,
    "Family D": 120
}

families_textarea = widgets.Textarea(
    value=json.dumps(initial_families_data, indent=2),
    description='Families (JSON):',
    layout={'width': '400px', 'height': '150px'}
)

liters_per_stroke_expert = widgets.IntText(value=7, description='Liters/Stroke:')
update_button = widgets.Button(description="Update Dashboard", button_style='info')

# --- Output Widgets and Tabs ---
chart_output = widgets.Output()
table_output = widgets.Output()
tab = widgets.Tab()
tab.children = [chart_output, table_output]
tab.set_title(0, 'Chart')
tab.set_title(1, 'Table')

def update_dashboard(b):
    """Reads data from widgets, performs calculations, and updates the tabbed outputs."""
    try:
        families_data = json.loads(families_textarea.value)
        lps = liters_per_stroke_expert.value
        if lps <= 0:
            raise ValueError("Liters per stroke must be positive.")
    except (json.JSONDecodeError, ValueError) as e:
        with chart_output:
            clear_output(wait=True)
            print(f"Error processing input: {e}")
        with table_output:
            clear_output(wait=True)
        return

    # --- Compute strokes needed using a dictionary comprehension ---
    strokes_needed = {family: math.ceil(liters / lps) for family, liters in families_data.items()}
    
    df = pd.DataFrame({
        'Family': list(strokes_needed.keys()),
        'Liters Needed': [families_data[key] for key in strokes_needed.keys()],
        'Strokes Required': list(strokes_needed.values())
    })

    # --- Update Plotly Chart ---
    with chart_output:
        clear_output(wait=True)
        fig = px.bar(
            df, 
            x='Family', 
            y='Strokes Required', 
            title='Pump Strokes Required Per Family',
            labels={'Strokes Required': 'Number of Strokes'},
            color='Liters Needed',
            hover_data=['Liters Needed']
        )
        fig.show()

    # --- Update Table ---
    with table_output:
        clear_output(wait=True)
        display(df.sort_values('Strokes Required', ascending=False))

# Link button and run initially
update_button.on_click(update_dashboard)
update_dashboard(None) # Initial run

# Display final layout
controls = widgets.VBox([families_textarea, liters_per_stroke_expert, update_button])
display(widgets.HBox([controls, tab]))